In [1]:
#Amkes Code umgeschrieben, da ich gerade nur die Inzidenz brauche und sie noch weitere Werte herausgezogen hat
import requests, json
import requests, json
from requests import get
from json import dumps
import datetime as DT

#Abfrage der Inzidenzen aus Deutschland

def get_data_by_id(id):
    url = "https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/rki_key_data_v/FeatureServer/0/query?"
    
    parameter = {
        'referer':'https://www.mywebapp.com',
        'user-agent':'python-requests/2.9.1',
        'where': f'AdmUnitId = {id}', # Welche landkreise sollen zurück gegeben werden
        'outFields': '*', # Rückgabe aller Felder
        'returnGeometry': False, # Keine Geometrien
        'f':'json', # Rückgabeformat, hier JSON
        'cacheHint': True # Zugriff über CDN anfragen
    }
    result3 = requests.get(url=url, params=parameter) #Anfrage absetzen
    resultjson3 = json.loads(result3.text) # Das Ergebnis JSON als Python Dictionary laden
    return float(resultjson3['features'][0]['attributes']['Inz7T'])

#Abfrage der Inzidenzen aus Großbritannien
#Antonias Code gekürzt auf das, was für die DB relevant ist

def get_incidence(city_id):
    url = "https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/rki_key_data_v/FeatureServer/0/query?"
    parameter = {
        'referer': 'https://www.mywebapp.com',
        'user-agent': 'python-requests/2.9.1',
        'where': f'AdmUnitId = {city_id}',  # Welche landkreise sollen zurück gegeben werden
        'outFields': '*',  # Rückgabe aller Felder
        'returnGeometry': False,  # Keine Geometrien
        'f': 'json',  # Rückgabeformat, hier JSON
        'cacheHint': True  # Zugriff über CDN anfragen
    }
    result = requests.get(url=url, params=parameter)  # Anfrage absetzen
    resultjson = json.loads(result.text)  # Das Ergebnis JSON als Python Dictionary laden
    incidence = resultjson['features'][0]['attributes']['Inz7T']
    return incidence

    # get_dailyCases Großbritanien

def get_dailyCases_uk (city_id, date):
    
    if city_id == "E12000007":
        AREA_TYPE = "region"
    else:
        AREA_TYPE = "ltla"
        
    ENDPOINT = "https://api.coronavirus.data.gov.uk/v1/data"
    AREA_CODE = city_id;
    DATE = date

    filters = [
         f"areaType={AREA_TYPE}",
         f"areaCode={AREA_CODE}",
         f"date={DATE}"

    ]

    structure = {

        "dailyCases" : "newCasesByPublishDate",
    }

    api_params = {
        "filters": str.join(";", filters),
        "structure": dumps(structure, separators=(",", ":")),
     
    }

    api_params["format"] = "json"
    response = get(ENDPOINT, params=api_params, timeout=10)
    assert response.status_code == 200, f"Failed request for {fmt}: {response.text}"
    json_content = response.json()
    return json_content["data"][0]["dailyCases"]
    
    # get_incidence_uk hier wird die 7-Tage Inzidenz für UK aus den DailyCases berechnet 

def get_incidence_uk (city_id, population):
    today = DT.date.today()
    date1 = today - DT.timedelta(days=1)
    date2 = today - DT.timedelta(days=2)
    date3 = today - DT.timedelta(days=3)
    date4 = today - DT.timedelta(days=4)
    date5 = today - DT.timedelta(days=5)
    date6 = today - DT.timedelta(days=6)
    date7 = today - DT.timedelta(days=7)

    day1 = get_dailyCases_uk(city_id, date1)
    day2 = get_dailyCases_uk(city_id, date2)
    day3 = get_dailyCases_uk(city_id, date3)
    day4 = get_dailyCases_uk(city_id, date4)
    day5 = get_dailyCases_uk(city_id, date5)
    day6 = get_dailyCases_uk(city_id, date6)
    day7 = get_dailyCases_uk(city_id, date7)
    
    dailyCasesSum = day1 + day2 + day3 + day4 + day5 + day6 + day7
    incidence = (dailyCasesSum / population) * 100000

    return round(incidence, 1)

In [2]:
#Speicherung der Inzidenzen aus Deutschland in Variablen
berlin_data= get_data_by_id(11)
berlin_mitte_data= get_data_by_id(11001)
dortmund_data= get_data_by_id(5913)
duesseldorf_data= get_data_by_id(5111)
essen_data= get_data_by_id(5113)
frankfurt_data= get_data_by_id(6412)
hamburg_data= get_data_by_id(2000)
koeln_data= get_data_by_id(5315)
muenchen_data= get_data_by_id(9162)
stuttgart_data= get_data_by_id(6412)
bremen_data= get_data_by_id(4011)
dresden_data= get_data_by_id(14612)
erfurt_data= get_data_by_id(16051)
hannover_data= get_data_by_id(3241)
kiel_data= get_data_by_id(1002)
magdeburg_data= get_data_by_id(15003)
mainz_data= get_data_by_id(7315)
potsdam_data= get_data_by_id(12054)
saarbruecken_data= get_data_by_id(10041)
schwerin_data= get_data_by_id(13004)
wiesbaden_data= get_data_by_id(6414)
#print(berlin_data)
berlin_mitte_data_string = json.dumps(berlin_mitte_data)
print(berlin_mitte_data_string)
berlin_mitte_data_float = float(berlin_mitte_data_string);
print(berlin_mitte_data_float)

#Speicherung der Inzidenzen aus Großbritannien in Variablen
manchester = get_incidence_uk("E08000003", 553230)
leeds = get_incidence_uk("E08000035", 792525)
glasgow =get_incidence_uk("S12000049", 1861315)
sheffield = get_incidence_uk("E08000019", 1569000)
nottingham = get_incidence_uk("E06000018", 331297)
newcastleUponTyne = get_incidence_uk("E08000021", 302820)
portsmouth = get_incidence_uk("E06000044", 1547000)
cardiff = get_incidence_uk("W06000015", 1097000)
belfast = get_incidence_uk("N09000003", 343000)
liverpool = get_incidence_uk("E08000012", 496784)
birmingham = get_incidence_uk("E08000025", 1149000)
edinburgh = get_incidence_uk("S12000036", 525000)
london = get_incidence_uk("E12000007", 8982000)
print(manchester)
print(belfast)

1657.3
1657.3
536.3
1512.8


In [9]:
#Verbindung zur Datenbank aufbauen
# Importing module
import mysql.connector
 
# Creating connection object
inzdb = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = "",
    db = "inzidenzen"
)
 
# Printing the connection object
print(inzdb)

In [7]:
#Einfügen des tagesaktuellen Inzidenzwerte der zehn größten Städte
#Tabelle Berlin-Mitte ist zum Testen
from datetime import datetime
from datetime import timedelta
from time import mktime
now = datetime.now()
current_date = now.strftime('%Y-%m-%d') #Ausgabe formatieren
cursor = inzdb.cursor()

#Einfügen der Inzidenzen aus Deutschland in die Datenbank

#cursor.execute("INSERT INTO `inzidenzen_deutschland_berlin_mitte`(`datum`, `inzidenz`) VALUES (NOW(), 355.5)")

#add_inzidenz_berlin_mitte="""INSERT INTO inzidenzen_deutschland_berlin_mitte (datum, inzidenz) VALUES(%s, %s)"""
#cursor.execute(add_inzidenz_berlin_mitte, ('2021-11-18', '1.1'))

add_inzidenz_berlin="""INSERT INTO inzidenzen_berlin (datum, inzidenz) VALUES(%s, %s)"""
cursor.execute(add_inzidenz_berlin, (current_date, berlin_data))

#add_inzidenz_berlin_mitte="""INSERT INTO inzidenzen_berlin_mitte (datum, inzidenz) VALUES(%s, %s)"""
#cursor.execute(add_inzidenz_berlin_mitte, (current_date, berlin_mitte_data))

add_inzidenz_hamburg="""INSERT INTO inzidenzen_hamburg (datum, inzidenz) VALUES(%s, %s)"""
cursor.execute(add_inzidenz_hamburg, (current_date, hamburg_data))

add_inzidenz_koeln="""INSERT INTO inzidenzen_koeln (datum, inzidenz) VALUES(%s, %s)"""
cursor.execute(add_inzidenz_koeln, (current_date, koeln_data))

add_inzidenz_muenchen="""INSERT INTO inzidenzen_muenchen (datum, inzidenz) VALUES(%s, %s)"""
cursor.execute(add_inzidenz_muenchen, (current_date, muenchen_data))

add_inzidenz_frankfurt="""INSERT INTO inzidenzen_frankfurt (datum, inzidenz) VALUES(%s, %s)"""
cursor.execute(add_inzidenz_frankfurt, (current_date, frankfurt_data))

add_inzidenz_duesseldorf="""INSERT INTO inzidenzen_duesseldorf (datum, inzidenz) VALUES(%s, %s)"""
cursor.execute(add_inzidenz_duesseldorf, (current_date, duesseldorf_data))

add_inzidenz_dortmund="""INSERT INTO inzidenzen_dortmund (datum, inzidenz) VALUES(%s, %s)"""
cursor.execute(add_inzidenz_dortmund, (current_date, dortmund_data))

add_inzidenz_stuttgart="""INSERT INTO inzidenzen_stuttgart (datum, inzidenz) VALUES(%s, %s)"""
cursor.execute(add_inzidenz_stuttgart, (current_date, stuttgart_data))

add_inzidenz_essen="""INSERT INTO inzidenzen_essen (datum, inzidenz) VALUES(%s, %s)"""
cursor.execute(add_inzidenz_essen, (current_date, essen_data))

add_inzidenz_bremen="""INSERT INTO inzidenzen_bremen (datum, inzidenz) VALUES(%s, %s)"""
cursor.execute(add_inzidenz_bremen, (current_date, bremen_data))

add_inzidenz_dresden="""INSERT INTO inzidenzen_dresden (datum, inzidenz) VALUES(%s, %s)"""
cursor.execute(add_inzidenz_dresden, (current_date, dresden_data))

add_inzidenz_erfurt="""INSERT INTO inzidenzen_erfurt (datum, inzidenz) VALUES(%s, %s)"""
cursor.execute(add_inzidenz_erfurt, (current_date, erfurt_data))

add_inzidenz_hannover="""INSERT INTO inzidenzen_hannover (datum, inzidenz) VALUES(%s, %s)"""
cursor.execute(add_inzidenz_hannover, (current_date, hannover_data))

add_inzidenz_kiel="""INSERT INTO inzidenzen_kiel (datum, inzidenz) VALUES(%s, %s)"""
cursor.execute(add_inzidenz_kiel, (current_date, kiel_data))

add_inzidenz_magdeburg="""INSERT INTO inzidenzen_magdeburg (datum, inzidenz) VALUES(%s, %s)"""
cursor.execute(add_inzidenz_magdeburg, (current_date, magdeburg_data))

add_inzidenz_mainz="""INSERT INTO inzidenzen_mainz (datum, inzidenz) VALUES(%s, %s)"""
cursor.execute(add_inzidenz_mainz, (current_date, mainz_data))

add_inzidenz_potsdam="""INSERT INTO inzidenzen_potsdam (datum, inzidenz) VALUES(%s, %s)"""
cursor.execute(add_inzidenz_potsdam, (current_date, potsdam_data))

add_inzidenz_saarbruecken="""INSERT INTO inzidenzen_saarbruecken (datum, inzidenz) VALUES(%s, %s)"""
cursor.execute(add_inzidenz_saarbruecken, (current_date, saarbruecken_data))

add_inzidenz_schwerin="""INSERT INTO inzidenzen_schwerin (datum, inzidenz) VALUES(%s, %s)"""
cursor.execute(add_inzidenz_schwerin, (current_date, schwerin_data))

add_inzidenz_wiesbaden="""INSERT INTO inzidenzen_wiesbaden (datum, inzidenz) VALUES(%s, %s)"""
cursor.execute(add_inzidenz_wiesbaden, (current_date, wiesbaden_data))

#Einfügen der Inzidenzen aus Großbritannien in die Datenbank

add_inzidenz_manchester="""INSERT INTO inzidenzen_manchester (datum, inzidenz) VALUES(%s, %s)"""
cursor.execute(add_inzidenz_manchester, (current_date, manchester))

add_inzidenz_leeds="""INSERT INTO inzidenzen_belfast (datum, inzidenz) VALUES(%s, %s)"""
cursor.execute(add_inzidenz_leeds, (current_date, leeds))

add_inzidenz_glasgow="""INSERT INTO inzidenzen_glasgow (datum, inzidenz) VALUES(%s, %s)"""
cursor.execute(add_inzidenz_glasgow, (current_date, glasgow))

add_inzidenz_sheffield="""INSERT INTO inzidenzen_sheffield (datum, inzidenz) VALUES(%s, %s)"""
cursor.execute(add_inzidenz_sheffield, (current_date, sheffield))

add_inzidenz_nottingham="""INSERT INTO inzidenzen_nottingham (datum, inzidenz) VALUES(%s, %s)"""
cursor.execute(add_inzidenz_nottingham, (current_date, nottingham))

add_inzidenz_newcastleUponTyne="""INSERT INTO inzidenzen_newcastleUponTyne (datum, inzidenz) VALUES(%s, %s)"""
cursor.execute(add_inzidenz_newcastleUponTyne, (current_date, newcastleUponTyne))

add_inzidenz_portsmouth="""INSERT INTO inzidenzen_portsmouth (datum, inzidenz) VALUES(%s, %s)"""
cursor.execute(add_inzidenz_portsmouth, (current_date, portsmouth))

add_inzidenz_cardiff="""INSERT INTO inzidenzen_cardiff (datum, inzidenz) VALUES(%s, %s)"""
cursor.execute(add_inzidenz_cardiff, (current_date, cardiff))

add_inzidenz_belfast="""INSERT INTO inzidenzen_belfast (datum, inzidenz) VALUES(%s, %s)"""
cursor.execute(add_inzidenz_belfast, (current_date, belfast))

add_inzidenz_liverpool="""INSERT INTO inzidenzen_liverpool (datum, inzidenz) VALUES(%s, %s)"""
cursor.execute(add_inzidenz_liverpool, (current_date, liverpool))

add_inzidenz_birmingham="""INSERT INTO inzidenzen_birmingham (datum, inzidenz) VALUES(%s, %s)"""
cursor.execute(add_inzidenz_birmingham, (current_date, birmingham))

add_inzidenz_edinburgh="""INSERT INTO inzidenzen_edinburgh (datum, inzidenz) VALUES(%s, %s)"""
cursor.execute(add_inzidenz_edinburgh, (current_date, edinburgh))

add_inzidenz_london="""INSERT INTO inzidenzen_london (datum, inzidenz) VALUES(%s, %s)"""
cursor.execute(add_inzidenz_london, (current_date, london))

cursor.close();
inzdb.commit()

#Datensätze auslesen
#cursor = inzdb.cursor()
#cursor.execute("select * from inzidenzen_deutschland_hamburg")
#cursor.execute("select * from inzidenzen_deutschland_koeln")
#cursor.execute("select * from inzidenzen_deutschland_muenchen")
#result = cursor.fetchall()
#cursor.close()

#for data in result:
#    print("Datum: " + str(data[0]) + "; Inzidenz: " + str(data[1]))

In [10]:
#Löschen von allen Werten, die mehr als 14 Tage zurückliegen

import datetime as d
from datetime import datetime
from datetime import timedelta
from time import mktime
now = datetime.now()
current_date = now.strftime('%Y-%m-%d') #Ausgabe formatieren
cursor = inzdb.cursor()

#print (d.date.today())
delete_date=d.date.today() - d.timedelta(7)
print("Es wurden alle Werte gelöscht, die vor diesem Datum liegen:")
print (delete_date)

#Löschen der Inzidenzen aus Deutschland, die älter als eine Woche sind

#delete_inzidenz_berlin_mitte="""DELETE FROM inzidenzen_berlin_mitte WHERE datum<%s"""
#cursor.execute(delete_inzidenz_berlin_mitte, (delete_date, ))

delete_inzidenz_berlin="""DELETE FROM inzidenzen_berlin WHERE datum<%s"""
cursor.execute(delete_inzidenz_berlin, (delete_date, ))

delete_inzidenz_dortmund="""DELETE FROM inzidenzen_dortmund WHERE datum<%s"""
cursor.execute(delete_inzidenz_dortmund, (delete_date, ))

delete_inzidenz_duesseldorf="""DELETE FROM inzidenzen_duesseldorf WHERE datum<%s"""
cursor.execute(delete_inzidenz_duesseldorf, (delete_date, ))

delete_inzidenz_essen="""DELETE FROM inzidenzen_essen WHERE datum<%s"""
cursor.execute(delete_inzidenz_essen, (delete_date, ))

delete_inzidenz_frankfurt="""DELETE FROM inzidenzen_frankfurt WHERE datum<%s"""
cursor.execute(delete_inzidenz_frankfurt, (delete_date, ))

delete_inzidenz_hamburg="""DELETE FROM inzidenzen_hamburg WHERE datum<%s"""
cursor.execute(delete_inzidenz_hamburg, (delete_date, ))

delete_inzidenz_koeln="""DELETE FROM inzidenzen_koeln WHERE datum<%s"""
cursor.execute(delete_inzidenz_koeln, (delete_date, ))

delete_inzidenz_muenchen="""DELETE FROM inzidenzen_muenchen WHERE datum<%s"""
cursor.execute(delete_inzidenz_muenchen, (delete_date, ))

delete_inzidenz_stuttgart="""DELETE FROM inzidenzen_stuttgart WHERE datum<%s"""
cursor.execute(delete_inzidenz_stuttgart, (delete_date, ))

delete_inzidenz_bremen="""DELETE FROM inzidenzen_bremen WHERE datum<%s"""
cursor.execute(delete_inzidenz_bremen, (delete_date, ))

delete_inzidenz_dresden="""DELETE FROM inzidenzen_dresden WHERE datum<%s"""
cursor.execute(delete_inzidenz_dresden, (delete_date, ))

delete_inzidenz_erfurt="""DELETE FROM inzidenzen_erfurt WHERE datum<%s"""
cursor.execute(delete_inzidenz_erfurt, (delete_date, ))

delete_inzidenz_hannover="""DELETE FROM inzidenzen_hannover WHERE datum<%s"""
cursor.execute(delete_inzidenz_hannover, (delete_date, ))

delete_inzidenz_kiel="""DELETE FROM inzidenzen_kiel WHERE datum<%s"""
cursor.execute(delete_inzidenz_kiel, (delete_date, ))

delete_inzidenz_magdeburg="""DELETE FROM inzidenzen_magdeburg WHERE datum<%s"""
cursor.execute(delete_inzidenz_magdeburg, (delete_date, ))

delete_inzidenz_mainz="""DELETE FROM inzidenzen_mainz WHERE datum<%s"""
cursor.execute(delete_inzidenz_mainz, (delete_date, ))

delete_inzidenz_potsdam="""DELETE FROM inzidenzen_potsdam WHERE datum<%s"""
cursor.execute(delete_inzidenz_potsdam, (delete_date, ))

delete_inzidenz_saarbruecken="""DELETE FROM inzidenzen_saarbruecken WHERE datum<%s"""
cursor.execute(delete_inzidenz_saarbruecken, (delete_date, ))

delete_inzidenz_schwerin="""DELETE FROM inzidenzen_schwerin WHERE datum<%s"""
cursor.execute(delete_inzidenz_schwerin, (delete_date, ))

delete_inzidenz_wiesbaden="""DELETE FROM inzidenzen_wiesbaden WHERE datum<%s"""
cursor.execute(delete_inzidenz_wiesbaden, (delete_date, ))

#Löschen der Inzidenzen aus Großbritannien, die älter als eine Woche sind

delete_inzidenz_manchester="""DELETE FROM inzidenzen_manchester WHERE datum<%s"""
cursor.execute(delete_inzidenz_manchester, (delete_date, ))

delete_inzidenz_leeds="""DELETE FROM inzidenzen_leeds WHERE datum<%s"""
cursor.execute(delete_inzidenz_leeds, (delete_date, ))

delete_inzidenz_glasgow="""DELETE FROM inzidenzen_glasgow WHERE datum<%s"""
cursor.execute(delete_inzidenz_glasgow, (delete_date, ))

delete_inzidenz_sheffield="""DELETE FROM inzidenzen_sheffield WHERE datum<%s"""
cursor.execute(delete_inzidenz_sheffield, (delete_date, ))

delete_inzidenz_nottingham="""DELETE FROM inzidenzen_nottingham WHERE datum<%s"""
cursor.execute(delete_inzidenz_nottingham, (delete_date, ))

delete_inzidenz_newcastleUponTyne="""DELETE FROM inzidenzen_newcastleUponTyne WHERE datum<%s"""
cursor.execute(delete_inzidenz_newcastleUponTyne, (delete_date, ))

delete_inzidenz_portsmouth="""DELETE FROM inzidenzen_portsmouth WHERE datum<%s"""
cursor.execute(delete_inzidenz_portsmouth, (delete_date, ))

delete_inzidenz_cardiff="""DELETE FROM inzidenzen_cardiff WHERE datum<%s"""
cursor.execute(delete_inzidenz_cardiff, (delete_date, ))

delete_inzidenz_belfast="""DELETE FROM inzidenzen_belfast WHERE datum<%s"""
cursor.execute(delete_inzidenz_belfast, (delete_date, ))

delete_inzidenz_liverpool="""DELETE FROM inzidenzen_liverpool WHERE datum<%s"""
cursor.execute(delete_inzidenz_liverpool, (delete_date, ))

delete_inzidenz_birmingham="""DELETE FROM inzidenzen_birmingham WHERE datum<%s"""
cursor.execute(delete_inzidenz_birmingham, (delete_date, ))

delete_inzidenz_edinburgh="""DELETE FROM inzidenzen_edinburgh WHERE datum<%s"""
cursor.execute(delete_inzidenz_edinburgh, (delete_date, ))

delete_inzidenz_london="""DELETE FROM inzidenzen_london WHERE datum<%s"""
cursor.execute(delete_inzidenz_london, (delete_date, ))

cursor.close()
inzdb.commit()

#Werte auslesen
#cursor = inzdb.cursor()
#cursor.execute("select * from inzidenzen_deutschland_berlin WHERE datum=2021-12-09")
#result = cursor.fetchall()
#cursor.close()

#for data in result:
#    print("Datum: " + str(data[0]) + "; Inzidenz: " + str(data[1]))

Löschen aller werte, die vor diesem Datum liegen:
2021-11-26
